## CS 445 Final Project: Model for Recognizing Emotions

In [ ]:
!pip install torchvision

In [ ]:
import numpy as np
import cv2
import scipy.linalg
import math
import os
%matplotlib inline
from matplotlib import pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimdef
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

In [ ]:
# # added by heather
!pip install gif2numpy
!pip install opencv-python
# from google.colab import drive
# drive.mount('/content/drive')

  Created wheel for gif2numpy: filename=gif2numpy-1.3-py2.py3-none-any.whl size=8391 sha256=3b5cd07faab2e069eef75f630091fd37fd5e9fc28d3159680a2d06e35d39c37c
  Stored in directory: /Users/rushalbutala/Library/Caches/pip/wheels/e0/9a/24/90d9287b2472b132fed8e91d85b3acd6ec39c9420d512120d6
  Created wheel for kaitaistruct: filename=kaitaistruct-0.9-py2.py3-none-any.whl size=5512 sha256=2dd496bfefd05f7e5e52e58e9759590db0988914929a7d321903ffc2f395b0d6
  Stored in directory: /Users/rushalbutala/Library/Caches/pip/wheels/da/13/6a/a51f1f909ab2ea9ac43db5616577860868944de1d45d2f098f
Successfully built gif2numpy kaitaistruct
  Using cached opencv_python-4.5.5.64-cp36-abi3-macosx_10_15_x86_64.whl (46.3 MB)


In [ ]:
import gif2numpy
import os
import random
# base_dir= "/Users/charl/Downloads/yalefaces/yalefaces/"
base_dir= "/Users/gauss/Downloads/Final Submission-2/yalefaces/new/"

dirs = os.listdir(base_dir)
N = len(dirs)
images = np.zeros(N)
image_labels = np.zeros(N)

for i in range(N):
    if dirs[i].endswith("happy.gif"):
        image_labels[i] = 1
    elif dirs[i].endswith("sad.gif"):
        image_labels[i] = 2

images = np.zeros((N, 243, 320, 3))
# This would print all the files and directories
idx = 0
for file in dirs:
    np_images, extensions, image_specs = gif2numpy.convert(base_dir+file)
    # print("type of image:", image, type(np_images[0]))
    # cv2_imshow(np_images[0])
    images[idx] = np_images[0]
    idx += 1

# images = images[:, :, :, 0]
# cv2.destroyWindow("np_image")

In [ ]:
# recognize faces
# square images
# base code from this source: https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81 

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
 
# Read the input images by accessing images[i] (from previous code cell)

dimensions = []
n = 0
gray_images = np.zeros((N, 243, 320))

for image in images :
    img = np.uint8(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_images[n] = gray

      # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
      # print(faces.shape)
      # Draw rectangle around the faces
    dimensions.append(faces)
    n += 1

maxi = 0

for faces in dimensions:
    for (x, y, w, h) in faces:
        if w > maxi:
            maxi = w
            
        if h > maxi:
            maxi = h

final_faces = np.zeros((N, maxi, maxi))   
i = 0

for faces in dimensions:
    for (x, y, w, h) in faces:
        final_faces[i] = cv2.resize(gray_images[i, y: y+ h,x : x + w], (maxi, maxi))
    
    i += 1
      # Display the output
      #  train_images[n]=img


size = int(N*0.8)

train_indices = random.sample(range(N), size)
test_indices = np.setdiff1d(range(N), train_indices)


#   idx = random.randint(0, len(gray_images))
#print(train_indices,test_indices)
train_images = final_faces[train_indices].astype(np.uint8)
test_images = final_faces[test_indices].astype(np.uint8)
train_labels = image_labels[train_indices]
test_labels = image_labels[test_indices]
print(train_images.shape)

(132, 170, 170)


In [ ]:
class FaceDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images.astype(np.float32)
        print(self.images.shape)
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [ ]:
classes = list(str(i) for i in range(3))
batch_size = 3


trainset = FaceDataset(images = train_images, labels = train_labels)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = FaceDataset(images = test_images, labels = test_labels)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)


(132, 170, 170)
(34, 170, 170)


In [ ]:
_, height, width = train_images.shape
kernel_size = 5
pool = 5
def calculate_shape(height, width, kernel_size, pool):
    in_height = int(((((height-kernel_size)+1)/pool-kernel_size)+1)/pool)
    in_width = int(((((width-kernel_size)+1)/pool-kernel_size)+1)/pool)
    return in_height * in_width
calculate_shape(height, width, kernel_size, pool)

25

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size) #gray scale image, single input channel
        self.pool = nn.MaxPool2d(pool, pool)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        self.fc1 = nn.Linear(16*calculate_shape(height, width, kernel_size, pool), 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


net = Net()

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [ ]:
# test model w/ new dataset
for epoch in range(10):  # loop over the dataset multiple times
        print(epoch)
        running_loss = 0.0
        
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            labels = labels.type(torch.LongTensor)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.unsqueeze(1)) # add one dimension to grayscale image
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

print('Finished Training')

0
1
2
3
4
5
6
7
8
9
Finished Training


In [ ]:
# output results & accuracy of model
PATH = './face_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images.unsqueeze(1))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 91 %
